In [1]:
# Load various RSMI-NE functions.

import rsmine.coarsegrainer.build_dataset as ds
from rsmine.coarsegrainer.cg_utils import *
from rsmine.coarsegrainer.analysis_utils import *

import polygons

/Users/efe/Documents/Research/RSMI-NE/rsmine/coarsegrainer/build_dataset.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
############# Load the definition of the graph and construct the reference graph

examples_dir = '.'

edges=np.loadtxt(os.path.join(examples_dir, 'quasiperiodic_data', 'edgedata.dat'))
nodes=np.loadtxt(os.path.join(examples_dir, 'quasiperiodic_data', 'nodedata.dat'), usecols=[0])
nodepos=np.loadtxt(os.path.join(examples_dir, 'quasiperiodic_data', 'nodedata.dat'), usecols=[1,2])
order_8 = np.loadtxt(os.path.join(examples_dir, 'quasiperiodic_data', 'order_of_8vtx.dat'))

G = construct_reference_graph(edges,nodes)

In [3]:
def rel_angle(vec, factor, vertex_type=None, ref_vec=None):
    """Returns the angle between vec and the reference vector.
    Used for rotating scaled elementary polygons.

    vertex_type is needed to determine the reference vector from the list
    """

    if ref_vec is None and vertex_type is not None:
        ref_vec = factor * nodepos[polygons.super_vertices[vertex_type][1][0]]

    dir = 'cw'
    rot = np.cross(vec, ref_vec)
    vdir = 'cw' if rot > 0 else 'ccw'

    r = (vec[0]*ref_vec[0]+vec[1]*ref_vec[1])/(np.linalg.norm(vec)*np.linalg.norm(ref_vec))

    angle = np.arccos(r)

    if vdir != dir:
        angle = 2*np.pi - angle

    if np.isnan(angle):
        return 0
    else:
        return angle

In [4]:
order_8[order_8[:, 1]==2]

array([[2.0000e+00, 2.0000e+00],
       [1.9600e+02, 2.0000e+00],
       [2.7400e+02, 2.0000e+00],
       [3.3450e+03, 2.0000e+00],
       [3.6160e+03, 2.0000e+00],
       [6.6170e+03, 2.0000e+00],
       [6.8110e+03, 2.0000e+00],
       [9.8890e+03, 2.0000e+00],
       [1.0160e+04, 2.0000e+00],
       [1.3161e+04, 2.0000e+00],
       [1.3355e+04, 2.0000e+00],
       [1.6433e+04, 2.0000e+00],
       [1.6704e+04, 2.0000e+00],
       [1.9705e+04, 2.0000e+00],
       [1.9899e+04, 2.0000e+00],
       [2.2977e+04, 2.0000e+00]])

In [5]:
############ Load the raw MC configurations (can take *a long while* for large files) #################

#Xs = iter_loadtxt('/home/cluster/sobisw/data/large_size/econfig_startseed%i_nsmaples50000.dat'%(EV_params['sample_no']),dtype=np.int8)

sample_no = 10705  # 10705 # 12121

sample_data_dir = os.path.join('data', 'configs') #os.path.join(os.pardir, 'data', 'configs')

Xs = iter_loadtxt(os.path.join(sample_data_dir,'econfig_startseed%i_nsmaples50000.dat'%(sample_no)), dtype=np.int8)

data_params = {}
data_params['N_samples'] = len(Xs)  # Set the correct number of samples loaded.
print("Data loaded")


Data loaded


In [664]:
# Set the various parameters of the system required by the code

# EV_params and data_params are generic data about samples of E,V

case_no= 51 # env, buffer, ll only come into play if case is set to "0"
V_index = 389  # 13554  # 389 is the central 8-vertex order 4 # 196 and 274 are 8-vertices of order 2 # 211 is 8-vertex of order 0
#V_order = order_8[V_index][1]
V_order = order_8[order_8[:, 0] == V_index][0, 1].astype(int)

print(V_order)

env_size = 8 # Not currently used by us
buffer_size = 2 # Not currently used by us
ll= 3 # Not currently used by us
disc=1


#We store these in a dictionary for experiment logging purposes only
EV_params = {
    'env_size' : env_size,
    'buffer_size': buffer_size,
    'block_size': ll, #assuming square block shape
    'sample_no': sample_no,
    'V_index': V_index,
    'V_order': V_order,
    'case_no':case_no
}

# These are needed by the generator
data_params = {
    'model': 'dimer_graph',
    'lattice_type': 'networkx',
    'L': 2048, # unused currently
    'N_samples': 1, # 1 is default. Set to actual value after generating or loading data.
    'dimension': 2,
}

4


In [665]:
# Instantiate the dataset object, using the loaded samples and the definition of the graph
# This will be used to generate the E,V splits.

generator=ds.dataset(**data_params, configurations = Xs, G=G, verbose=True)
#generator=dataset(**data_params, configurations = Xs, G=G, verbose=True)
print(data_params)
print(EV_params)

{'model': 'dimer_graph', 'lattice_type': 'networkx', 'L': 2048, 'N_samples': 1, 'dimension': 2}
{'env_size': 8, 'buffer_size': 2, 'block_size': 3, 'sample_no': 10705, 'V_index': 389, 'V_order': 4, 'case_no': 51}


In [666]:
# Print the shape of Xs (num_samples,num_variables_in_graph):
Xs.shape

(50000, 51800)

In [667]:
# Double check what sample parameters are currently selected:
EV_params['sample_no'],EV_params['V_index'],EV_params['V_order']

(10705, 389, 4)

In [668]:
# Load definitions of membranes, regions

regions_data_dir = examples_dir + '/quasiperiodic_data/regions/'

if V_order != -1:
    mem_edges0_ids = eids_from_edges(edges, np.loadtxt(
        regions_data_dir + 'reg_mem_edgelist_svtx%i_sord%i_mem0.dat' % (EV_params['V_index'], EV_params['V_order']), dtype=int))
elif V_order == -1:
    mem_edges0_ids = []
    mem_edges1_ids = []
    mem_edges2_ids = []
    mem_edges3_ids = []
    mem_edges4_ids = []

elif V_order == -1:
    mem_edges0_ids = []
    mem_edges1_ids = []
    mem_edges2_ids = []
    mem_edges3_ids = []
    mem_edges4_ids = []
    
if V_order == 0:
    mem_edges1_ids = []
    mem_edges2_ids = []
    mem_edges3_ids = []
    mem_edges4_ids = []
elif V_order == 1:
    mem_edges1_ids = eids_from_edges(edges, np.loadtxt(
        regions_data_dir + 'reg_mem_edgelist_svtx%i_sord%i_mem1.dat' % (EV_params['V_index'], EV_params['V_order']), dtype=int))
    mem_edges2_ids = []
    mem_edges3_ids = []
    mem_edges4_ids = []
elif V_order == 2:
    mem_edges1_ids = eids_from_edges(edges, np.loadtxt(
        regions_data_dir + 'reg_mem_edgelist_svtx%i_sord%i_mem1.dat' % (EV_params['V_index'], EV_params['V_order']), dtype=int))
    mem_edges2_ids = eids_from_edges(edges, np.loadtxt(
        regions_data_dir + 'reg_mem_edgelist_svtx%i_sord%i_mem2.dat' % (EV_params['V_index'], EV_params['V_order']), dtype=int))
    mem_edges3_ids = []
    mem_edges4_ids = []
elif V_order > 2:
    mem_edges1_ids = eids_from_edges(edges, np.loadtxt(
        regions_data_dir + 'reg_mem_edgelist_svtx%i_sord%i_mem1.dat' % (EV_params['V_index'], EV_params['V_order']), dtype=int))
    mem_edges2_ids = eids_from_edges(edges, np.loadtxt(
        regions_data_dir + 'reg_mem_edgelist_svtx%i_sord%i_mem2.dat' % (EV_params['V_index'], EV_params['V_order']), dtype=int))
    mem_edges3_ids = eids_from_edges(edges, np.loadtxt(
        regions_data_dir + 'reg_mem_edgelist_svtx%i_sord%i_mem3.dat' % (EV_params['V_index'], EV_params['V_order']), dtype=int))
    mem_edges4_ids = eids_from_edges(edges, np.loadtxt(
        regions_data_dir + 'reg_mem_edgelist_svtx%i_sord%i_mem4.dat' % (EV_params['V_index'], EV_params['V_order']), dtype=int))

if V_order != -1:
    reg_edges0_ids = eids_from_edges(edges, np.loadtxt(
        regions_data_dir + 'reg_edgelist_svtx%i_sord%i_mem0.dat' % (EV_params['V_index'], EV_params['V_order']), dtype=int))
elif V_order == -1:
    reg_edges0_ids = []
    reg_edges1_ids = []
    reg_edges2_ids = []
    reg_edges3_ids = []
    reg_edges4_ids = []

if V_order == 0:
    reg_edges1_ids = []
    reg_edges2_ids = []
    reg_edges3_ids = []
    reg_edges4_ids = []
elif V_order == 1:
    reg_edges1_ids = eids_from_edges(edges, np.loadtxt(
        regions_data_dir + 'reg_edgelist_svtx%i_sord%i_mem1.dat' % (EV_params['V_index'], EV_params['V_order']), dtype=int))
    reg_edges2_ids = []
    reg_edges3_ids = []
    reg_edges4_ids = []
elif V_order == 2:
    reg_edges1_ids = eids_from_edges(edges, np.loadtxt(
        regions_data_dir + 'reg_edgelist_svtx%i_sord%i_mem1.dat' % (EV_params['V_index'], EV_params['V_order']), dtype=int))
    reg_edges2_ids = eids_from_edges(edges, np.loadtxt(
        regions_data_dir + '/reg_edgelist_svtx%i_sord%i_mem2.dat' % (EV_params['V_index'], EV_params['V_order']), dtype=int))
    reg_edges3_ids = []
    reg_edges4_ids = []
elif V_order > 2:
    reg_edges1_ids = eids_from_edges(edges, np.loadtxt(
        regions_data_dir + 'reg_edgelist_svtx%i_sord%i_mem1.dat' % (EV_params['V_index'], EV_params['V_order']), dtype=int))
    reg_edges2_ids = eids_from_edges(edges, np.loadtxt(
        regions_data_dir + '/reg_edgelist_svtx%i_sord%i_mem2.dat' % (EV_params['V_index'], EV_params['V_order']), dtype=int))
    reg_edges3_ids = eids_from_edges(edges, np.loadtxt(
        regions_data_dir + '/reg_edgelist_svtx%i_sord%i_mem3.dat' % (EV_params['V_index'], EV_params['V_order']), dtype=int))
    reg_edges4_ids = eids_from_edges(edges, np.loadtxt(
        regions_data_dir + '/reg_edgelist_svtx%i_sord%i_mem4.dat' % (EV_params['V_index'], EV_params['V_order']), dtype=int))

ball4, _ = construct_VE_edgelists(G, EV_params['V_index'], L_B=1, ll=4, cap=8)
ball5, _ = construct_VE_edgelists(G, EV_params['V_index'], L_B=1, ll=5, cap=8)
ball8, _ = construct_VE_edgelists(G, EV_params['V_index'], L_B=1, ll=8, cap=11)
ball10, _ = construct_VE_edgelists(G, EV_params['V_index'], L_B=1, ll=10, cap=15)
ball12, _ = construct_VE_edgelists(G, EV_params['V_index'], L_B=1, ll=12, cap=15)
ball16, _ = construct_VE_edgelists(G, EV_params['V_index'], L_B=1, ll=16, cap=19)
ball20, _ = construct_VE_edgelists(G, EV_params['V_index'], L_B=1, ll=20, cap=26)
ball24, _ = construct_VE_edgelists(G, EV_params['V_index'], L_B=1, ll=24, cap=26)
ball32, _ = construct_VE_edgelists(G, EV_params['V_index'], L_B=1, ll=32, cap=35)
ball40, _ = construct_VE_edgelists(G, EV_params['V_index'], L_B=1, ll=40, cap=50)
ball48, _ = construct_VE_edgelists(G, EV_params['V_index'], L_B=1, ll=48, cap=50)
ball54, _ = construct_VE_edgelists(G, EV_params['V_index'], L_B=1, ll=54, cap=60)
ball64, _ = construct_VE_edgelists(G, EV_params['V_index'], L_B=1, ll=64, cap=70)
ball68, _ = construct_VE_edgelists(G, EV_params['V_index'], L_B=1, ll=68, cap=70)
ball72, _ = construct_VE_edgelists(G, EV_params['V_index'], L_B=1, ll=72, cap=84)
ball84, _ = construct_VE_edgelists(G, EV_params['V_index'], L_B=1, ll=84, cap=90)


# Construct lists of edges defining the selected E,V if ***NOT*** using the cases, but rather topological balls.
# If using the cases, it's much safer to use the "multiple case" path, even if the case list of length 1.

GV_edges, GE_edges = construct_VE_edgelists(G, V_index, L_B=buffer_size, ll=ll, cap=ll+buffer_size+env_size)

In [669]:
# Define the E,V areas via "cases" instead (more appropriate for non-spherical regions, like empires etc.)

#### Be super careful!!! IF the variables  reg_.... is initialized from some previous vertex,
#### there won't be an error, ven of lower order vertex doesn't have this membrane. 


cGV_edges = {}
cGE_edges = {}

"""     # Case #1
#auxGV_edges = sorted(set(reg_edges0_ids))
#auxGE_edges = sorted(set(reg_edges1_ids) -set(reg_edges0_ids) - set(mem_edges0_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[1] = auxGV_edges
#cGE_edges[1] = auxGE_edges

    # Case #2
#auxGV_edges = sorted(set(reg_edges0_ids))
#auxGE_edges = sorted(set(reg_edges1_ids) | set(mem_edges1_ids) -set(reg_edges0_ids) - set(mem_edges0_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[2] = auxGV_edges
#cGE_edges[2] = auxGE_edges

#     # Case #3
# auxGV_edges = sorted(set(reg_edges0_ids))
# auxGE_edges = sorted(set(reg_edges2_ids) -set(reg_edges0_ids) - set(mem_edges0_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
# cGV_edges[3] = auxGV_edges
# cGE_edges[3] = auxGE_edges

#     # Case #4
# auxGV_edges = sorted(set(reg_edges0_ids))
# auxGE_edges = sorted(set(reg_edges2_ids) -set(reg_edges1_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
# cGV_edges[4] = auxGV_edges
# cGE_edges[4] = auxGE_edges

#     # Case #5
# auxGV_edges = sorted(set(reg_edges0_ids) | set(mem_edges0_ids))
# auxGE_edges = sorted(set(reg_edges2_ids) -set(reg_edges1_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
# cGV_edges[5] = auxGV_edges
# cGE_edges[5] = auxGE_edges

#     # Case #6
# auxGV_edges = sorted(set(reg_edges0_ids) | set(mem_edges0_ids))
# auxGE_edges = sorted(set(reg_edges2_ids) -set(reg_edges1_ids) - set(mem_edges1_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
# cGV_edges[6] = auxGV_edges
# cGE_edges[6] = auxGE_edges

    # Case #7
#auxGV_edges = sorted(set(reg_edges0_ids) | set(mem_edges0_ids))
#auxGE_edges = sorted(set(ball32) -set(reg_edges2_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[7] = auxGV_edges
#cGE_edges[7] = auxGE_edges

    # Case #8
auxGV_edges = sorted(set(reg_edges0_ids) | set(mem_edges0_ids))
auxGE_edges = sorted(set(ball32) -set(reg_edges2_ids) - set(mem_edges2_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
cGV_edges[8] = auxGV_edges
cGE_edges[8] = auxGE_edges

    # Case #9
#auxGV_edges = sorted(set(reg_edges1_ids))
#auxGE_edges = sorted(set(reg_edges2_ids) -set(reg_edges1_ids) - set(mem_edges1_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[9] = auxGV_edges
#cGE_edges[9] = auxGE_edges

    # Case #10
#auxGV_edges = sorted(set(reg_edges1_ids))
#auxGE_edges = sorted(set(reg_edges2_ids) | set(mem_edges2_ids) -set(reg_edges1_ids) - set(mem_edges1_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[10] = auxGV_edges
#cGE_edges[10] = auxGE_edges

#     # Case #11
# auxGV_edges = sorted(set(reg_edges1_ids))
# auxGE_edges = sorted(set(ball32) -set(reg_edges1_ids) - set(mem_edges1_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
# cGV_edges[11] = auxGV_edges
# cGE_edges[11] = auxGE_edges

#     # Case #12
# auxGV_edges = sorted(set(reg_edges1_ids))
# auxGE_edges = sorted(set(ball32) -set(reg_edges2_ids) - set(mem_edges2_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
# cGV_edges[12] = auxGV_edges
# cGE_edges[12] = auxGE_edges

    # Case #13
#auxGV_edges = sorted(set(reg_edges1_ids))
#auxGE_edges = sorted(set(ball32) -set(reg_edges2_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[13] = auxGV_edges
#cGE_edges[13] = auxGE_edges

    # Case #14
#auxGV_edges = sorted(set(reg_edges1_ids) | set(mem_edges1_ids))
#auxGE_edges = sorted(set(ball32) -set(reg_edges2_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[14] = auxGV_edges
#cGE_edges[14] = auxGE_edges

    # Case #15
#auxGV_edges = sorted(set(reg_edges1_ids) | set(mem_edges1_ids))
#auxGE_edges = sorted(set(ball32) -set(reg_edges2_ids) -set(mem_edges2_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[15] = auxGV_edges
#cGE_edges[15] = auxGE_edges

#     # Case #16
# auxGV_edges = sorted(set(reg_edges2_ids))
# auxGE_edges = sorted(set(ball32) -set(reg_edges2_ids) -set(mem_edges2_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
# cGV_edges[16] = auxGV_edges
# cGE_edges[16] = auxGE_edges

    # Case #17
#auxGV_edges = sorted(set(reg_edges2_ids))
#auxGE_edges = sorted(set(ball48) -set(reg_edges2_ids) -set(mem_edges2_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[17] = auxGV_edges
#cGE_edges[17] = auxGE_edges

#     # Case #18 was is ball 20 or ball 24????
# auxGV_edges = sorted(set(reg_edges2_ids))
# auxGE_edges = sorted(set(ball48) -set(ball24))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
# cGV_edges[18] = auxGV_edges
# cGE_edges[18] = auxGE_edges

#     # Case #19
# auxGV_edges = sorted(set(reg_edges0_ids))
# auxGE_edges = sorted(set(ball16) -set(reg_edges1_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
# cGV_edges[19] = auxGV_edges
# cGE_edges[19] = auxGE_edges

    # Case #20
#auxGV_edges = sorted(set(reg_edges0_ids))
#auxGE_edges = sorted(set(ball16) -set(reg_edges1_ids) -set(mem_edges1_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[20] = auxGV_edges
#cGE_edges[20] = auxGE_edges

    # Case #21
#auxGV_edges = sorted(set(reg_edges0_ids) | set(mem_edges0_ids))
#auxGE_edges = sorted(set(ball16) -set(reg_edges1_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[21] = auxGV_edges
#cGE_edges[21] = auxGE_edges

    # Case #22
#auxGV_edges = sorted(set(reg_edges0_ids) | set(mem_edges0_ids))
#auxGE_edges = sorted(set(ball16) -set(reg_edges1_ids) -set(mem_edges1_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[22] = auxGV_edges
#cGE_edges[22] = auxGE_edges

    # Case #23
#auxGV_edges = sorted(set(reg_edges1_ids))
#auxGE_edges = sorted(set(ball16) -set(reg_edges1_ids) -set(mem_edges1_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[23] = auxGV_edges
#cGE_edges[23] = auxGE_edges

    # Case #24
#auxGV_edges = sorted(set(reg_edges1_ids))
#auxGE_edges = sorted(set(ball24) -set(ball16))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[24] = auxGV_edges
#cGE_edges[24] = auxGE_edges

    # Case #25
#auxGV_edges = sorted(set(reg_edges1_ids) | set(mem_edges1_ids))
#auxGE_edges = sorted(set(ball24) -set(ball16))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[25] = auxGV_edges
#cGE_edges[25] = auxGE_edges

    # Case #26
#auxGV_edges = sorted(set(reg_edges1_ids) | set(mem_edges1_ids))
#auxGE_edges = sorted(set(ball16) -set(ball12))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[26] = auxGV_edges
#cGE_edges[26] = auxGE_edges

    # Case #27
#auxGV_edges = sorted(set(reg_edges2_ids))
#auxGE_edges = sorted(set(ball32) -set(ball24))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[27] = auxGV_edges
#cGE_edges[27] = auxGE_edges


    # Case #28
#auxGV_edges = sorted(set(reg_edges0_ids))
#auxGE_edges = sorted(set(ball8) - set(reg_edges0_ids) - set(mem_edges0_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[28] = auxGV_edges
#cGE_edges[28] = auxGE_edges


    # Case #29
#auxGV_edges = sorted(set(reg_edges0_ids))
#auxGE_edges = sorted(set(ball12) - set(reg_edges0_ids) - set(mem_edges0_ids))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[29] = auxGV_edges
#cGE_edges[29] = auxGE_edges


     #Case #30
#auxGV_edges = sorted(set(reg_edges0_ids))
#auxGE_edges = sorted(set(ball12) - set(ball5))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[30] = auxGV_edges
#cGE_edges[30] = auxGE_edges

    # Case #31
#auxGV_edges = sorted(set(reg_edges0_ids))
#auxGE_edges = sorted(set(ball16) - set(ball5))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[31] = auxGV_edges
#cGE_edges[31] = auxGE_edges

    # Case #32
#auxGV_edges = sorted(set(reg_edges0_ids) | set(mem_edges0_ids))
#auxGE_edges = sorted(set(ball16) - set(ball5))#  #sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
#cGV_edges[32] = auxGV_edges
#cGE_edges[32] = auxGE_edges

# # Case #33
# auxGV_edges = sorted(set(reg_edges2_ids))
# auxGE_edges = sorted(
#     set(ball48) - set(reg_edges3_ids) - set(mem_edges3_ids))
# cGV_edges[33] = auxGV_edges
# cGE_edges[33] = auxGE_edges

## Case #34
#auxGV_edges = sorted(set(reg_edges2_ids))
#auxGE_edges = sorted(
#    set(reg_edges4_ids) - set(reg_edges3_ids) - set(mem_edges3_ids))
#cGV_edges[34] = auxGV_edges
#cGE_edges[34] = auxGE_edges

# # Case #35
# auxGV_edges = sorted(set(reg_edges1_ids))
# auxGE_edges = sorted(
#     set(reg_edges3_ids) - set(reg_edges2_ids) - set(mem_edges2_ids))
# cGV_edges[35] = auxGV_edges
# cGE_edges[35] = auxGE_edges

# Case #36
auxGV_edges = sorted(set(reg_edges0_ids))
# sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
auxGE_edges = sorted(set(ball12) -
                        set(reg_edges0_ids) - set(mem_edges0_ids))
cGV_edges[36] = auxGV_edges
cGE_edges[36] = auxGE_edges

# Case #37
auxGV_edges = sorted(set(reg_edges0_ids))
# sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
auxGE_edges = sorted(set(ball12) - set(ball4))
cGV_edges[37] = auxGV_edges
cGE_edges[37] = auxGE_edges

# Case #38
# buffer width 2-3 case for level 1
auxGV_edges = sorted(set(reg_edges1_ids))
auxGE_edges = sorted(
    set(ball32) - set(ball8))
cGV_edges[38] = auxGV_edges
cGE_edges[38] = auxGE_edges

# Case #39
# buffer width 3-4 case for level 1
auxGV_edges = sorted(set(reg_edges1_ids))
auxGE_edges = sorted(
    set(ball32) - set(ball12))
cGV_edges[39] = auxGV_edges
cGE_edges[39] = auxGE_edges

# Case #40
# buffer width 3-4 case for level 1
auxGV_edges = sorted(set(reg_edges1_ids))
auxGE_edges = sorted(
    set(ball24) - set(ball12))
cGV_edges[40] = auxGV_edges
cGE_edges[40] = auxGE_edges

   # Case #41
# buffer width 2
# used for 3-vertices at sigma^2=level-0 (indices: 399, 404)
auxGV_edges = sorted(set(reg_edges0_ids) | set(mem_edges0_ids))
# sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
auxGE_edges = sorted(set(ball12) - set(ball5))
cGV_edges[41] = auxGV_edges
cGE_edges[41] = auxGE_edges

# Case #42
# buffer width 3-4
# used for 3-vertices at sigma^3=level-1 (indices: 429, 447)
auxGV_edges = sorted(set(reg_edges1_ids) | set(mem_edges1_ids))
# sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
auxGE_edges = sorted(set(ball32) - set(ball10))
cGV_edges[42] = auxGV_edges
cGE_edges[42] = auxGE_edges

# Case #43
# buffer width
# used for 8-vertices at sigma^3=level-1 (indices: 389)
auxGV_edges = sorted(set(reg_edges2_ids))
# sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
auxGE_edges = sorted(set(ball48) - set(ball20))
cGV_edges[43] = auxGV_edges
cGE_edges[43] = auxGE_edges

# Case #44
auxGV_edges = sorted(set(ball5))
auxGE_edges = sorted(set(ball32) - set(ball8))
cGV_edges[44] = auxGV_edges
cGE_edges[44] = auxGE_edges

# Case #45
auxGV_edges = sorted(set(ball8))
auxGE_edges = sorted(set(ball48) - set(ball16))
cGV_edges[45] = auxGV_edges
cGE_edges[45] = auxGE_edges

# Case #46
# used for 3-vertices at sigma^1 (doesn't correspond to any level) (indices: 393)
auxGV_edges, auxGE_edges = construct_VE_edgelists(
    G, EV_params['V_index'], L_B=2, ll=2, cap=16)
auxGV_edges = list(set(auxGV_edges) - set([807, 877, 879, 46284, 46289, 793]))
cGV_edges[46] = auxGV_edges
cGE_edges[46] = auxGE_edges


# Case #47
# use for 8-vertices at sigma^4=level-2 (index:389)
auxGV_edges = sorted(set(reg_edges3_ids))
# sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
auxGE_edges = sorted(set(ball72) - set(ball48))
cGV_edges[47] = auxGV_edges
cGE_edges[47] = auxGE_edges

# Case #48
# use for 8-vertices at sigma^4=level-2 (index:389)
auxGV_edges = sorted(set(ball16))
# sorted(set(reg_edges2_ids)-(set(reg_edges0_ids) | set(mem_edges0_ids)))
auxGE_edges = sorted(set(ball48) - set(ball24))
cGV_edges[48] = auxGV_edges
cGE_edges[48] = auxGE_edges



# Case #49
# use for 8-vertices at sigma^2=level-0 (index:389) (perfect tile)
auxGV_edges = sorted(set(reg_edges1_ids))
auxGV_edges = list(set(auxGV_edges) -
                   set([13980, 13982,
                        26928, 26930,
                        39876, 39878,
                        7506, 7508,
                        868, 870,
                        20454, 20456,
                        876, 878,
                        33402, 33404]) -
                   set([13979, 26927, 39875, 7505, 867, 20453, 875, 33401]))
_, auxGE_edges = construct_VE_edgelists(
    G, EV_params['V_index'], L_B=3, ll=4, cap=24)
cGV_edges[49] = auxGV_edges
cGE_edges[49] = auxGE_edges



# Case #50
# use for 3-vertices at sigma^2=level-0 (index:404, 399, ...) (perfect tile)
vertex_type = '3fold'
sigma_scale = 2
factor = polygons.silver_ratio ** (sigma_scale-1)
#angle = np.arccos(
#    np.dot(nodepos[EV_params['V_index']], factor*nodepos[polygons.super_vertices[vertex_type][1][0]]) /
#    (np.linalg.norm(nodepos[EV_params['V_index']])*np.linalg.norm(factor*nodepos[polygons.super_vertices[vertex_type][1][0]])))
angle = rel_angle(nodepos[EV_params['V_index']],
                  factor, vertex_type=vertex_type)

auxGV_edges = polygons.rotate_and_scale_Vgraph(
    angle, sigma_scale, vertex_type)[0]
# TODO: can try increasing the buffer size here
auxGE_edges = construct_VE_edgelists(G, V_index, L_B=4, ll=4, cap=20)[1]
cGV_edges[50] = auxGV_edges
cGE_edges[50] = auxGE_edges
"""



# Case 51
# Use for 8-vertices at sigma^4=level-2 (index:389) (perfect tile)
vertex_type = '8fold'
sigma_scale = 4
factor = polygons.silver_ratio ** (sigma_scale-1)

auxGV_edges = polygons.scale_Vgraph(sigma_scale, vertex_type)[0]
# TODO: determine the proper buffer size here
#auxGE_edges = construct_VE_edgelists(G, V_index, L_B=8, ll=16, cap=48)[1]
#auxGE_edges = sorted(set(ball72) - set(ball54))
auxGE_edges = sorted(set(ball64) - set(ball40))

cGV_edges[51] = auxGV_edges
cGE_edges[51] = auxGE_edges


"""
# Case 52
# Use for 3-vertices at sigma^4=level-2 (index:196, 274, ...) (perfect tile)
vertex_type = '3fold'
sigma_scale = 4
factor = polygons.silver_ratio ** (sigma_scale-1)
angle = rel_angle(nodepos[EV_params['V_index']],
                  factor, vertex_type=vertex_type)

auxGV_edges = polygons.rotate_and_scale_Vgraph(
    angle, sigma_scale, vertex_type)[0]
# TODO: determine the proper buffer size here
#auxGE_edges = construct_VE_edgelists(G, V_index, L_B=8, ll=16, cap=48)[1]
auxGE_edges = sorted(set(ball64) - set(ball40))
cGV_edges[52] = auxGV_edges
cGE_edges[52] = auxGE_edges




# Case 53
# Use for 8-vertices at sigma^3=level-1 (index:389) (perfect tile)
vertex_type = '8fold'
sigma_scale = 3
factor = polygons.silver_ratio ** (sigma_scale-1)

auxGV_edges = polygons.scale_Vgraph(sigma_scale, vertex_type)[0]
auxGE_edges = sorted(set(ball48) - set(ball24))
cGV_edges[53] = auxGV_edges
cGE_edges[53] = auxGE_edges



# Case 54
# Use for 3-vertices at sigma^4=level-2 (index:196, 274, ...) (perfect tile)
vertex_type = '3fold'
sigma_scale = 3
factor = polygons.silver_ratio ** (sigma_scale-1)
angle = rel_angle(nodepos[EV_params['V_index']],
                  factor, vertex_type=vertex_type)

auxGV_edges = polygons.rotate_and_scale_Vgraph(
    angle, sigma_scale, vertex_type)[0]
# TODO: determine the proper buffer size here
#auxGE_edges = construct_VE_edgelists(G, V_index, L_B=8, ll=16, cap=48)[1]
auxGE_edges = sorted(set(ball48) - set(ball20))
cGV_edges[54] = auxGV_edges
cGE_edges[54] = auxGE_edges
"""


"\n# Case 52\n# Use for 3-vertices at sigma^4=level-2 (index:196, 274, ...) (perfect tile)\nvertex_type = '3fold'\nsigma_scale = 4\nfactor = polygons.silver_ratio ** (sigma_scale-1)\nangle = rel_angle(nodepos[EV_params['V_index']],\n                  factor, vertex_type=vertex_type)\n\nauxGV_edges = polygons.rotate_and_scale_Vgraph(\n    angle, sigma_scale, vertex_type)[0]\n# TODO: determine the proper buffer size here\n#auxGE_edges = construct_VE_edgelists(G, V_index, L_B=8, ll=16, cap=48)[1]\nauxGE_edges = sorted(set(ball64) - set(ball40))\ncGV_edges[52] = auxGV_edges\ncGE_edges[52] = auxGE_edges\n\n\n\n\n# Case 53\n# Use for 8-vertices at sigma^3=level-1 (index:389) (perfect tile)\nvertex_type = '8fold'\nsigma_scale = 3\nfactor = polygons.silver_ratio ** (sigma_scale-1)\n\nauxGV_edges = polygons.scale_Vgraph(sigma_scale, vertex_type)[0]\nauxGE_edges = sorted(set(ball48) - set(ball24))\ncGV_edges[53] = auxGV_edges\ncGE_edges[53] = auxGE_edges\n\n\n\n# Case 54\n# Use for 3-vertices at

In [670]:
# Double check which cases you selected:
list(cGV_edges.keys())

[51]

In [671]:
#####################################################
############ Generate E,V datasets ##################
#####################################################

EV_dir = sample_data_dir = os.path.join(os.pardir, 'data', 'EandV')


In [672]:
############ Generate E,V datasets ***FOR A SINGLE CASE/custom definition of E,V*** 

Vs, Es = generator.rsmi_data_graph(GV_edges,GE_edges)

print(round((sys.getsizeof(np.array(Es)) + sys.getsizeof(np.array(Vs)))/ 1024 / 1024, 2), " MB")



np.save(os.path.join(EV_dir, r"Es_%i_vi%i_e%ib%ill%i" % (EV_params['sample_no'],EV_params['V_index'],EV_params['env_size'],EV_params['buffer_size'],EV_params['block_size'])),Es)
np.save(os.path.join(EV_dir, r"Vs_%i_vi%i_e%ib%ill%i" % (EV_params['sample_no'], EV_params['V_index'], EV_params['env_size'], EV_params['buffer_size'], EV_params['block_size'])), Vs)

np.save(os.path.join(EV_dir, r"Es_%i_vi%i_c%i" % (EV_params['sample_no'], EV_params['V_index'], case_no)), Es)
np.save(os.path.join(EV_dir, r"Vs_%i_vi%i_c%i" % (EV_params['sample_no'], EV_params['V_index'], case_no)), Vs)



Preparing the RSMI dataset...
RSMI dataset prepared.
137.33  MB


In [673]:
############ Generate E,V datasets ***FOR MULTIPLE CASES*** 

case_list = list(cGV_edges.keys())
for case_no in case_list:
    auxVs, auxEs = generator.rsmi_data_graph(cGV_edges[case_no],cGE_edges[case_no])
    print("Case_no: ",case_no,", Size: ",round((sys.getsizeof(np.array(auxEs)) + sys.getsizeof(np.array(auxVs)))/ 1024 / 1024, 2), " MB")
    np.save(os.path.join(EV_dir, r"Es_%i_vi%i_c%i" % (EV_params['sample_no'],EV_params['V_index'],case_no)), auxEs)
    np.save(os.path.join(EV_dir, r"Vs_%i_vi%i_c%i" % (EV_params['sample_no'],EV_params['V_index'],case_no)), auxVs)

Preparing the RSMI dataset...
RSMI dataset prepared.
Case_no:  51 , Size:  2760.32  MB
